In [ ]:
!pip install huggingface-hub
!pip install datasets
!pip install nltk

In [98]:
import nltk
nltk.download('punkt')
import re
from torch.utils.data import DataLoader
from datasets import load_dataset
import datasets
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import wordnet
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

[nltk_data] Downloading package punkt to /home/minne/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/minne/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
dataset = load_dataset('daily_dialog',split='train')

Using custom data configuration default
Reusing dataset daily_dialog (/home/minne/.cache/huggingface/datasets/daily_dialog/default/1.0.0/c03444008e9508b8b76f1f6793742d37d5e5f83364f8d573c2747bff435ea55c)


In [28]:
dataset

Dataset({
    features: ['dialog', 'act', 'emotion'],
    num_rows: 11118
})

In [130]:
df_train = dataset.to_pandas()

In [132]:
df_train['dialog'][0]

array(['Say , Jim , how about going for a few beers after dinner ? ',
       ' You know that is tempting but is really not good for our fitness . ',
       ' What do you mean ? It will help us to relax . ',
       " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ",
       " I guess you are right.But what shall we do ? I don't feel like sitting at home . ",
       ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ',
       " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ",
       ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . ',
       " Good.Let ' s go now . ", ' All right . '], dtype=object)

In [153]:
stp_w = stopwords.words('english')
def rm_stop_words(tokens):
    clean = []
    for w in tokens:
        if w not in stp_w:
            clean.append(w)
    return clean

In [190]:
def text_normalization(text_list, wo_stop_words = True):
    list_lower = [re.sub(r'[^a-z0-9\s]',' ',text.lower()) for text in text_list] #removes punctuation, converts all letters to lowercase
    tokenized_list = [nltk.word_tokenize(sent) for sent in list_lower]
    if wo_stop_words:
        for i, tokens in enumerate(tokenized_list):
            tokenized_list[i] = rm_stop_words(tokens)
    lema = wordnet.WordNetLemmatizer()
    for i, tokens in enumerate(tokenized_list): 
        tags_list = pos_tag(tokens, tagset = None)
        lema_words = []
        for token, pos_token in tags_list:
            if pos_token.startswith('V'): #verb
                pos_val = 'v'
            elif pos_token.startswith('J'): #adj
                pos_val = 'a'
            elif pos_token.startswith('R'): #adv
                pos_val = 'r'
            else:
                pos_val = 'n' #noun
            lema_token = lema.lemmatize(token, pos_val)
            lema_words.append(lema_token)
        tokenized_list[i] = ' '.join(lema_words)  
    return tokenized_list

In [191]:
text_normalization(df_train['dialog'][0])[0]

'say jim go beer dinner'

Meaning of some sentances lost, if I remove stop words, so I will keep all words.

Should I remove numbers from the text?

In [198]:
df_train['dialog_prepros'] = df_train['dialog'].apply(text_normalization)

In [200]:
df_train['dialog_prepros'][0]

['say jim go beer dinner',
 'know tempt really good fitness',
 'mean help u relax',
 'really think make u fat act silly remember last time',
 'guess right shall feel like sit home',
 'suggest walk gym play singsong meet friend',
 'good idea hear mary sally often go play pingpong perhaps make foursome',
 'sound great willing could ask go dance u excellent exercise fun',
 'good let go',
 'right']

In [201]:
df_train['dialog_prepros_w_stop'] = df_train['dialog'].apply(lambda x:text_normalization(x, False))

In [202]:
df_train['dialog_prepros_w_stop'][0]

['say jim how about go for a few beer after dinner',
 'you know that be tempt but be really not good for our fitness',
 'what do you mean it will help u to relax',
 'do you really think so i don t it will just make u fat and act silly remember last time',
 'i guess you be right but what shall we do i don t feel like sit at home',
 'i suggest a walk over to the gym where we can play singsong and meet some of our friend',
 'that s a good idea i hear mary and sally often go there to play pingpong perhaps we can make a foursome with them',
 'sound great to me if they be willing we could ask them to go dance with u that be excellent exercise and fun too',
 'good let s go now',
 'all right']

In [204]:
df_train.to_excel('DailyDialog_lemmatized.xlsx', )

In [211]:
df_train['dialog_prepros_w_stop'][0]

['say jim how about go for a few beer after dinner',
 'you know that be tempt but be really not good for our fitness',
 'what do you mean it will help u to relax',
 'do you really think so i don t it will just make u fat and act silly remember last time',
 'i guess you be right but what shall we do i don t feel like sit at home',
 'i suggest a walk over to the gym where we can play singsong and meet some of our friend',
 'that s a good idea i hear mary and sally often go there to play pingpong perhaps we can make a foursome with them',
 'sound great to me if they be willing we could ask them to go dance with u that be excellent exercise and fun too',
 'good let s go now',
 'all right']

In [241]:
tfidf = TfidfVectorizer()

In [300]:
tf_idf_transformed = tfidf.fit(df_train['dialog_prepros_w_stop'].apply(lambda x:', '.join(x)))

In [287]:
#tf_idf_transformed = df_train['dialog_prepros_w_stop'].apply(lambda x: list(tfidf.transform(x)))

In [274]:
#dataset.with_transform()
df_train['tf_idf_transformed']

0        [  (0, 10719)\t0.2555721735467611\n  (0, 6870)...
1        [  (0, 13768)\t0.14918103724167428\n  (0, 1300...
2        [  (0, 13768)\t0.12281680582247476\n  (0, 1357...
3        [  (0, 13768)\t0.2915987198626801\n  (0, 10427...
4        [  (0, 12363)\t0.24305856600691172\n  (0, 1120...
                               ...                        
11113    [  (0, 13772)\t0.17021991242860476\n  (0, 1105...
11114    [  (0, 13768)\t0.13729790117857787\n  (0, 1082...
11115    [  (0, 12867)\t0.4559664189024807\n  (0, 12387...
11116    [  (0, 13768)\t0.1888655140677948\n  (0, 11184...
11117    [  (0, 12327)\t0.21255588081950755\n  (0, 9400...
Name: tf_idf_transformed, Length: 11118, dtype: object